In [23]:
! brew install p7zip

==> Downloading https://homebrew.bintray.com/bottles/p7zip-16.02_1.mojave.bottle
==> Downloading from https://akamai.bintray.com/bb/bb715042a9067df735cd7d032a159
######################################################################## 100.0%
==> Pouring p7zip-16.02_1.mojave.bottle.tar.gz
🍺  /usr/local/Cellar/p7zip/16.02_1: 103 files, 4.7MB
==> `brew cleanup` has not been run in 30 days, running now...
Removing: /Users/shuang/Library/Logs/Homebrew/gettext... (64B)
Removing: /Users/shuang/Library/Logs/Homebrew/pcre2... (64B)
Removing: /Users/shuang/Library/Logs/Homebrew/git... (64B)


In [31]:
from pathlib import Path
import subprocess
import os

In [25]:
raw_datadir = Path('../data/raw/')

In [26]:
filepath = raw_datadir / 'biology.stackexchange.com.7z'
print(filepath)

../data/raw/biology.stackexchange.com.7z


In [27]:
def get_output_dirname(path):
    filename = path.name
    return path.with_suffix('')
    
def open_7z(path):
    output_dirname = get_output_dirname(path)
    output_dirname.mkdir(exist_ok=True, parents=True)
    exe_str = f"7za e {path} -o{output_dirname}"
    
    print(f'Reading data from {path}')
    print(f'Saving data to {output_dirname}')
    print("Running:", exe_str)
    p = subprocess.call(exe_str, shell=True)
    print(f'Done. {p}')
    return 

In [28]:
open_7z(filepath)

Reading data from ../data/raw/biology.stackexchange.com.7z
Saving data to ../data/raw/biology.stackexchange.com
Running: 7za e ../data/raw/biology.stackexchange.com.7z -o../data/raw/biology.stackexchange.com
Done. 0


In [29]:
!ls ../data/raw/biology.stackexchange.com

Badges.xml      PostHistory.xml Posts.xml       Users.xml
Comments.xml    PostLinks.xml   Tags.xml        Votes.xml


In [32]:
file_list = os.listdir(raw_datadir)
for file_name in file_list:
    if os.path.splitext(file_name)[1] == '.7z':
        print (file_name)
        path = raw_datadir / file_name
        open_7z(path)


math.stackexchange.com.7z
Reading data from ../data/raw/math.stackexchange.com.7z
Saving data to ../data/raw/math.stackexchange.com
Running: 7za e ../data/raw/math.stackexchange.com.7z -o../data/raw/math.stackexchange.com
Done. 0
chemistry.stackexchange.com.7z
Reading data from ../data/raw/chemistry.stackexchange.com.7z
Saving data to ../data/raw/chemistry.stackexchange.com
Running: 7za e ../data/raw/chemistry.stackexchange.com.7z -o../data/raw/chemistry.stackexchange.com
Done. 0
physics.stackexchange.com.7z
Reading data from ../data/raw/physics.stackexchange.com.7z
Saving data to ../data/raw/physics.stackexchange.com
Running: 7za e ../data/raw/physics.stackexchange.com.7z -o../data/raw/physics.stackexchange.com
Done. 0
biology.stackexchange.com.7z
Reading data from ../data/raw/biology.stackexchange.com.7z
Saving data to ../data/raw/biology.stackexchange.com
Running: 7za e ../data/raw/biology.stackexchange.com.7z -o../data/raw/biology.stackexchange.com
Done. 134


In [35]:
import xml.etree.ElementTree as ET
import pandas as pd

def parse_xml_file(path):
    """Read the xml file from path and return a pandas dataframe with all the data."""
    root = ET.parse(path).getroot()
    rows = root.findall('row')
    return pd.DataFrame([dict(row.items()) for row in rows])

In [43]:
math = parse_xml_file('../data/raw/math.stackexchange.com/Comments.xml')

In [44]:
physics = parse_xml_file('../data/raw/physics.stackexchange.com/Comments.xml')
biology = parse_xml_file('../data/raw/physics.stackexchange.com/Comments.xml')
chemistry = parse_xml_file('../data/raw/chemistry.stackexchange.com/Comments.xml')

In [45]:
df = pd.concat([math, physics, biology, chemistry])

In [46]:
df.to_csv("../data/csv/all.csv", index = False)

In [50]:
math.to_csv("../data/csv/math.csv", index = False)

In [48]:
physics.to_csv("../data/csv/physics.csv", index = False)

In [49]:
biology.to_csv("../data/csv/biology.csv", index = False)